In [ ]:
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, TextAreaField # BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError # Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta
from itsdangerous import URLSafeTimedSerializer, SignatureExpired
from werkzeug.exceptions import HTTPException
from logging import FileHandler, WARNING

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
file_handler = FileHandler('errorlog.txt')
file_handler.setLevel(WARNING)
app.logger.addHandler(file_handler)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = "login"

class UserForm(FlaskForm):
	gender = StringField("gender")
	age = StringField("age")
	scripts_played = StringField("scripts_played")
	types = StringField("types")
	scripts_want = StringField("scripts_want")
	district = StringField("district")
	submit = SubmitField("Submit")

class User(db.Model, UserMixin):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    password = db.Column(db.String(255))
    name = db.Column(db.String(255))
    username = db.Column(db.String(255), unique=True)
    district = db.Column(db.String(120))
    scripts_want = db.Column(db.String(120))
    types = db.Column(db.String(120))
    scripts_played = db.Column(db.String(120))
    age = db.Column(db.String(120))
    gender = db.Column(db.String(120))
    active = db.Column(db.Boolean())
    confirmed_at = db.Column(db.DateTime())
    admin = db.Column(db.Boolean)
    replies = db.relationship('Reply', backref='user', lazy='dynamic')
    apponitments = db.relationship('Appointment', backref='user', lazy='dynamic')

class NewThread(FlaskForm):
    title = StringField('Title')
    description = StringField('Description')
    submit1 = SubmitField('发布约本')

class NewReply(FlaskForm):
    message = TextAreaField('Message')

class Thread(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(30))
    description = db.Column(db.String(200))
    date_created = db.Column(db.DateTime())

    replies = db.relationship('Reply', backref='thread', lazy='dynamic')

    def last_post_date(self):
        last_reply = Reply.query.filter_by(thread_id=self.id).order_by(Reply.id.desc()).first()

        if last_reply:
            return last_reply.date_created

        return self.date_created

class Appointment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    scripts = db.Column(db.String(30))
    venues = db.Column(db.String(30))
    date = db.Column(db.String(30))
    time = db.Column(db.String(30))
    num = db.Column(db.String(30))
    lack = db.Column(db.String(30))
    tel = db.Column(db.String(30))
    date_created = db.Column(db.DateTime())

class NewAppointment(FlaskForm):
    scripts = StringField('scripts')
    venues = StringField('venues')
    date = StringField('date')
    time = StringField('time')
    num = StringField('num')
    lack = StringField('lack')
    tel = StringField('tel')
    submit2 = SubmitField('提交预约')

class Contact(db.Model):
    __tablename__ = 'contact'
    id = db.Column(db.Integer, primary_key=True)
    venuename = db.Column(db.String(30))
    venueemail = db.Column(db.String(30))
    venuetel = db.Column(db.String(30))
    description = db.Column(db.String(2000))
    date_created = db.Column(db.DateTime())

class Update(db.Model):
    __tablename__ = 'update'
    id = db.Column(db.Integer, primary_key=True)
    venueid = db.Column(db.Integer)
    activity = db.Column(db.String(30))
    message = db.Column(db.String(2000))
    date_created = db.Column(db.DateTime())

class Reply(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    thread_id = db.Column(db.Integer, db.ForeignKey('thread.id'))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    message = db.Column(db.String(200))
    date_created = db.Column(db.DateTime())

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route('/home')
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact', methods=["GET", 'POST'])
def contact():
    if request.method == "POST":
        venuename = request.form['venuename']
        venueemail = request.form['venueemail']
        venuetel = request.form['venuetel']
        description = request.form['description']
        new_contact = Contact(venuename=venuename,venueemail=venueemail,venuetel=venuetel,description=description)
        try:
            db.session.add(new_contact)
            db.session.commit()
            flash("发送成功!")
            return render_template("index.html")
        except Exception as e:
            print("Exception occurred.",e)
            flash("发送失败!")
            return '发送失败'
    else:
        return render_template('contact.html')

@app.route('/update', methods=["GET", 'POST'])
def update():
    if request.method == "POST":
        venueid = request.form['venueid']
        activity = request.form['activity']
        message = request.form['message']
        new_update = Update(venueid=venueid,activity=activity,message=message)
        try:
            db.session.add(new_update)
            db.session.commit()
            flash("发送成功!")
            print("发送成功!")
            return redirect(url_for('home'))
        except Exception as e:
            flash("发送失败!")
            print("Exception occurred.",e)
            return '发送失败'
    else:
        return render_template('update.html')
    
@app.route('/login', methods=["GET", 'POST'])
def login():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if not user:
            print("Wrong username or username not found.")
            flash("错误的用户名或未找到用户名")
            return redirect(url_for('login'))
        else:
            if bcrypt.check_password_hash(user.password, password):
                print("Successfully logged in.")
                login_user(user)
                flash("登陆成功")
                return redirect(url_for('dashboard'))
            else:
                print("Wrong password.")
                flash("错误的密码.")
                return redirect(url_for('login'))
    else:
        return render_template('login.html')

@app.route('/dashboard', methods=['GET', 'POST'])
@login_required
def dashboard():
	form = UserForm()
	id = current_user.id
	name_to_update = User.query.get_or_404(id)
	if request.method == "POST":
		name_to_update.gender = request.form['gender']
		name_to_update.age = request.form['age']
		name_to_update.scripts_played = request.form['scripts_played']
		name_to_update.types = request.form['types']
		name_to_update.scripts_want = request.form['scripts_want']
		name_to_update.district = request.form['district']
		try:
			db.session.commit()
			flash("User Updated Successfully!")
			return render_template("dashboard.html",
				form=form,
				name_to_update = name_to_update)
		except:
			flash("Error!  Looks like there was a problem...try again!")
			return render_template("dashboard.html",
				form=form,
				name_to_update = name_to_update)
	else:
		return render_template("dashboard.html",
				form=form,
				name_to_update = name_to_update,
				id = id)

	return render_template('dashboard.html')

@app.route('/my_appointments', methods=['GET','POST'])
@login_required
def my_appointments():
    if current_user.admin:
        appointments = Appointment.query.all()
    else:
        appointments = Appointment.query.filter_by(user=current_user).all()
    return render_template('my_appointments.html', appointments=appointments)

@app.route("/venues")
def venues():
    sqlalchemyObj = db.engine.execute('select * from venues')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[0]
        dict['rating'] = i[2]
        dict['Price'] = i[4]
        dict['Url'] = i[13]
        dict['Img_url'] = i[1]
        dict['District'] = i[5]
        dict['Event'] = i[9]
        dict['Adress'] = i[12]
        dict['Tel'] = i[16]
        dataset.append(dict.copy())

    sqlalchemyObj = db.engine.execute('select * from wzmap')
    wzmap = []
    for i in sqlalchemyObj:
        wzmap.append(i)
    # print(venues)
    dataset6 = []
    dict={}
    for i in wzmap:
        dict['Name'] = i[1]
        dict['Lat'] = i[3]
        dict['Lng'] = i[2]
        # print(i)
        # print(dict)
        dataset6.append(dict.copy()) #markers.append(fld.copy())
    return render_template('venues.html', title='Venues',dataset=dataset,entries3 = dataset6)


@app.route("/products")
def products():
    return render_template('products.html', title='products')

@app.route("/scripts_5")
def scripts_5():
    sqlalchemyObj = db.engine.execute('select * from scripts_5')
    scripts_5 = []
    for i in sqlalchemyObj:
        scripts_5.append(i)
    dataset1 = []
    dict={}
    for i in scripts_5:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset1.append(dict.copy())

    return render_template('scripts_5.html', title='scripts_5', dataset1=dataset1)

@app.route("/scripts_6")
def scripts_6():
    sqlalchemyObj = db.engine.execute('select * from scripts_6')
    scripts_6 = []
    for i in sqlalchemyObj:
        scripts_6.append(i)
    dataset2 = []
    dict={}
    for i in scripts_6:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset2.append(dict.copy())

    return render_template('scripts_6.html', title='scripts_6', dataset2=dataset2)

@app.route("/scripts_7")
def scripts_7():
    sqlalchemyObj = db.engine.execute('select * from scripts_7')
    scripts_7 = []
    for i in sqlalchemyObj:
        scripts_7.append(i)
    dataset3 = []
    dict={}
    for i in scripts_7:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset3.append(dict.copy())

    return render_template('scripts_7.html', title='scripts_7', dataset3=dataset3)

@app.route("/scripts_8")
def scripts_8():
    sqlalchemyObj = db.engine.execute('select * from scripts_8')
    scripts_8 = []
    for i in sqlalchemyObj:
        scripts_8.append(i)
    dataset4 = []
    dict={}
    for i in scripts_8:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset4.append(dict.copy())

    return render_template('scripts_8.html', title='scripts_8', dataset4=dataset4)

@app.route("/scripts_others")
def scripts_others():
    sqlalchemyObj = db.engine.execute('select * from scripts_others')
    scripts_others = []
    for i in sqlalchemyObj:
        scripts_others.append(i)
    dataset5 = []
    dict={}
    for i in scripts_others:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset5.append(dict.copy())

    return render_template('scripts_others.html', title='scripts_others', dataset5=dataset5)

@app.route("/scripts_chengxian")
def scripts_chengxian():
    sqlalchemyObj = db.engine.execute('select * from scripts_chengxian ORDER BY Num')
    scripts_chengxian = []
    for i in sqlalchemyObj:
        scripts_chengxian.append(i)
    dataset6 = []
    dict={}
    for i in scripts_chengxian:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dict['Num'] = i[4]
        dataset6.append(dict.copy())

    return render_template('scripts_chengxian.html', title='scripts_chengxian', dataset6=dataset6)

@app.route("/scripts_qinggan")
def scripts_qinggan():
    sqlalchemyObj = db.engine.execute('select * from scripts_qinggan ORDER BY Num')
    scripts_qinggan = []
    for i in sqlalchemyObj:
        scripts_qinggan.append(i)
    dataset7 = []
    dict={}
    for i in scripts_qinggan:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Num'] = i[4]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset7.append(dict.copy())

    return render_template('scripts_qinggan.html', title='scripts_qinggan', dataset7=dataset7)

@app.route("/scripts_yinghe")
def scripts_yinghe():
    sqlalchemyObj = db.engine.execute('select * from scripts_yinghe ORDER BY Num')
    scripts_yinghe = []
    for i in sqlalchemyObj:
        scripts_yinghe.append(i)
    dataset8 = []
    dict={}
    for i in scripts_yinghe:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Num'] = i[4]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset8.append(dict.copy())

    return render_template('scripts_yinghe.html', title='scripts_yinghe', dataset8=dataset8)

@app.route("/scripts_huanle")
def scripts_huanle():
    sqlalchemyObj = db.engine.execute('select * from scripts_huanle ORDER BY Num')
    scripts_huanle = []
    for i in sqlalchemyObj:
        scripts_huanle.append(i)
    dataset9 = []
    dict={}
    for i in scripts_huanle:
        dict['Name'] = i[0]
        dict['Info'] = i[2]
        dict['Typ'] = i[5]
        dict['Num'] = i[4]
        dict['Img_url'] = i[1]
        dict['Writer'] = i[3]
        dict['Desc'] = i[6]
        dataset9.append(dict.copy())

    return render_template('scripts_huanle.html', title='scripts_huanle', dataset9=dataset9)

@app.route('/blog', methods=['GET', 'POST'])
def blog():
    blog_form = NewThread()
    appo_form = NewAppointment()
    if blog_form.submit1.data and blog_form.validate_on_submit():
        new_thread = Thread(title=blog_form.title.data, description=blog_form.description.data, date_created=datetime.now())
        db.session.add(new_thread)
        db.session.commit()
        return redirect(url_for('blog'))


    if appo_form.submit2.data and appo_form.validate_on_submit():
        new_appointment = Appointment(user_id=current_user.id, scripts=appo_form.scripts.data, venues=appo_form.venues.data, date =appo_form.date.data,time =appo_form.time.data, num =appo_form.num.data, lack =appo_form.lack.data, tel=appo_form.tel.data,date_created=datetime.now())
        db.session.add(new_appointment)
        db.session.commit()
        return redirect(url_for('blog'))

    appointments = Appointment.query.all()
    threads = Thread.query.all()

    return render_template('blog.html', appo_form=appo_form, blog_form=blog_form, threads=threads, appointments=appointments, current_user=current_user)


@app.route('/thread/<thread_id>', methods=['GET', 'POST'])
def thread(thread_id):
    form = NewReply()

    thread = Thread.query.get(int(thread_id))

    if form.validate_on_submit():
        reply = Reply(user_id=current_user.id, message=form.message.data, date_created=datetime.now())
        thread.replies.append(reply)
        db.session.commit()

    replies = Reply.query.filter_by(thread_id=thread_id).all()

    return render_template('thread.html', thread=thread, form=form, replies=replies, current_user=current_user)

@app.route('/register', methods=['GET','POST'])
def register():
    status = "register"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        password = bcrypt.generate_password_hash(password)
        name = request.form['name']
        gender = request.form['gender']
        age = request.form['age']
        scripts_played = request.form['scripts_played']
        scripts_want = request.form['scripts_want']
        district = request.form['district']
        types = request.form['types']
        new_user = User(name=name,username=username,email=email,password=password,types=types,district=district, scripts_want=scripts_want, scripts_played=scripts_played,age=age,gender=gender,admin=0)
        try:
            db.session.add(new_user)
            db.session.commit()
            print("注册成功!")
            return redirect(url_for('login'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html')

@app.route('/logout', methods=["GET","POST"])
def logout():
    logout_user()
    return redirect(url_for('home'))

@app.errorhandler(Exception)
def handle_exception(e):
    # pass through HTTP errors
    if isinstance(e, HTTPException):
        f = open('errorlog.txt', "a")
        f.write("Code:"+str(e.code)+"\n")
        f.write("Name:"+e.name+"\n")
        f.write("Description:"+e.description+"\n")
        f.write("\n")
        f.close()
        return e
    title = "Non-HTTP Exceptions"
    # now you're handling non-HTTP exceptions only
    return render_template("500_generic.html", title=title, e=e), 500

if __name__ == '__main__':
    app.run('localhost', 3020
           )

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\Estela\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
C:\Users\Estela\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:3020/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 10:50:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:50:39] "GET /blog HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:50:52] "GET /venues HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:50:53] "GET /None HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 10:50:53] "GET /None HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 10:50:55] "GET /img/footer-img/footer-widgets-bg.png HTTP/1.1" 404 -
12

Successfully logged in.


127.0.0.1 - - [13/Dec/2021 10:51:55] "GET /blog HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:52:03] "GET /my_appointments HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:52:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:52:08] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:52:13] "GET /update HTTP/1.1" 200 -
